### LLM z-bench 测试
下面，我们使用真格资金建立的一套中文语言模型的测试集，来对这个模型进行测试：
https://github.com/zhenbench/z-bench

真格基金提供了3个级别的测试集，基础能力、进阶能力和垂直能力，并且提供了使用 GPT3.5 和 GPT4 的参考答案，可以用这个测试集进行问答测试，并跟GPT的结果进行比较，来检验语言模型的能力。

使用 SageMaker通过Endpoint Name来调用大语言模型。

In [2]:
# endpoint_name = "mt-chatglm2-6b-ft-g4dn"
# endpoint_name = "mt-chatglm2-g4dn"
endpoint_name = "mt-cpmbee-g4dn"
model_code="cpmbee"

In [3]:
import json
import boto3

client = boto3.client('runtime.sagemaker')
def query_endpoint(encoded_json):
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/json', Body=encoded_json)
    model_predictions = json.loads(response['Body'].read())
    generated_text = model_predictions["outputs"]
    return generated_text



In [10]:
# from prompt_generator import generate_prompt as gen_prompt
import prompt_generator as pg
import importlib

importlib.reload(pg)

<module 'prompt_generator' from '/home/ec2-user/SageMaker/LLM_Workshop/prompt_generator.py'>

#### 基础能力
首先来看看基础能力：

In [5]:
import pandas as pd
benchdf_common = pd.read_csv("./z-bench/common.samples.csv")

In [6]:
benchdf_common.shape

63

In [22]:
%%time

import random
r = random.randint(0, len(benchdf_common))
test_data = benchdf_common.iloc[r]

payload = pg.create_payload(test_data["Prompt"], model_code)
response = query_endpoint(json.dumps(payload).encode('utf-8'))

print("任务类型:", test_data["任务类型"], "参考回答:", test_data["参考回答"])
print('Prompt:', test_data["Prompt"])
print(model_code, response)
print("GPT3.5:", test_data["GPT3.5"])
print("GPT4:", test_data["GPT4"])

任务类型: `代码` `字符串编写` 参考回答: nan
Prompt: # Python 3.7def randomly_split_dataset(folder, filename, split_ratio=[0.8, 0.2]):   df = pd.read_json(folder + filename, lines=True)   train_name, test_name = "train.jsonl", "test.jsonl"   df_train, df_test = train_test_split(df, test_size=split_ratio[1], random_state=42)   df_train.to_json(folder + train_name, orient='records', lines=True)   df_test.to_json(folder + test_name, orient='records', lines=True)randomly_split_dataset('finetune_data/', 'dataset.jsonl')   # An elaborate, high quality docstring for the above function:"""
cpmbee [{'question': '# Python 3.7def randomly_split_dataset(folder, filename, split_ratio=[0.8, 0.2]):   df = pd.read_json(folder + filename, lines=True)   train_name, test_name = "train.jsonl", "test.jsonl"   df_train, df_test = train_test_split(df, test_size=split_ratio[1], random_state=42)   df_train.to_json(folder + train_name, orient=\'records\', lines=True)   df_test.to_json(folder + test_name, orient=\'records\', li

In [21]:
response

[{'question': '请判断这些句子中表达的情绪：我的猫好可爱♥♥', '<ans>': '好评'}]

#### 进阶能力

In [24]:
import pandas as pd
benchdf_emergent = pd.read_csv("./z-bench/emergent.samples.csv")

In [25]:
benchdf_emergent.head()

,Prompt,任务类型,参考回答,GPT3.5,GPT4
0,如果我在空中垂直抛球，球最初向哪个方向行进？,`物理直觉`,向上,当你在空中垂直抛球时，球最初行进的方向是向上的，也就是和地面垂直的方向。在球到达最高点时，它...,当你在空中垂直向上抛球时，球最初的方向是向上。它将沿着垂直方向受到重力的作用而减速，直至在其...
1,金元素属于哪一种化学键？,`物理直觉`,金属键,金是一种金属元素，因此通常会形成金属键。金属键是由金属元素组成的化学键，这些金属元素的原子之...,金属元素，如金（Au），通常与其他金属元素通过金属键（metallic bond）相互连接。...
2,假设一个原子发射一个光子，原子会发生什么？,`物理直觉`,失去能量,当一个原子发射一个光子时，它会处于一个激发态或高能态，并且会向低能态或基态释放能量。在这个过...,当一个原子发射一个光子时，这意味着原子中的一个电子从一个较高的能级跃迁到一个较低的能级。在这...
3,假设一辆车可以在 3.85s 的时间内从 0 加速到 27.8 m/s，请计算这辆车的加速度...,`物理`,7.22 m/s/s,根据运动学公式，加速度等于速度变化量除以时间。那么，这辆车的加速度为：a = (27.8 m...,为了计算加速度，我们可以使用以下公式：加速度（a）= (最终速度（vf） - 初始速度（vi...
4,Zdeno Chara 以 175.1 km/h 的速度创造了最快的冰球拍击记录。假设一个冰...,`物理`,a) 8.28 kg·m/s b) Chara 的拍击速度为 175.1 km/h = 48...,a) Chara 的拍击球的动量为：p = mv其中，m = 0.17 kg 是冰球的质量，...,a) 要计算冰球的动量，我们使用动量公式：p = m * v，其中 p 为动量，m 为质量，...


In [28]:
import random
r = random.randint(0, len(benchdf_emergent))
test_data = benchdf_emergent.iloc[r]


payload = pg.create_payload(test_data["Prompt"], model_code)
response = query_endpoint(json.dumps(payload).encode('utf-8'))

print("任务类型:", test_data["任务类型"], "参考回答:", test_data["参考回答"])
print('Prompt:', test_data["Prompt"])
print(model_code, response)
print("GPT3.5:", test_data["GPT3.5"])
print("GPT4:", test_data["GPT4"])


任务类型: `分类` `相关推荐` 参考回答: nan
Prompt: 帮我推荐些类似《哈利波特》《原则》《跨越鸿沟》《追风筝的人》这样的书
cpmbee [{'question': '帮我推荐些类似《哈利波特》《原则》《跨越鸿沟》《追风筝的人》这样的书', '<ans>': '小说'}]
GPT3.5: 以下是一些类似于《哈利波特》、《原则》、《跨越鸿沟》、《追风筝的人》的书籍推荐：《摆渡人》- 克莱儿·麦克福尔《小王子》- 圣埃克苏佩里《傲慢与偏见》- 简·奥斯汀《1984》- 乔治·奥威尔《人类简史》- 尤瓦尔·赫拉利《飘》- 玛格丽特·米切尔《了不起的盖茨比》- F. 斯科特·菲茨杰拉德《麦田里的守望者》- J·D·塞林格《万历十五年》- 黄仁宇《福尔摩斯探案全集》- 亚瑟·柯南·道尔希望能对您有所帮助！
GPT4: 以下是一些建议的书籍，涵盖了魔幻小说、人生哲学、社会科学和小说等多种类型：环游地球80天（Around the World in 80 Days）- 儒勒·凡尔纳巨人的陨落（Fall of Giants）- 肯·福莱特杀手代号47（The Power of Habit）- 查尔斯·杜希格雪崩：心灵的觉醒（The Alchemist）- 保罗·科贝略驯龙高手（How to Train Your Dragon）- 克雷斯·柯达尔金色笼子（The Golden Cage）- 琪琪·塞尔福人类简史（Sapiens: A Brief History of Humankind）- 尤瓦尔·赫拉利自私的基因（The Selfish Gene）- 理查德·道金斯五人行（The Five People You Meet in Heaven）- 米奇·阿尔班福尔摩斯探案全集（The Complete Sherlock Holmes）- 阿瑟·柯南·道尔时间简史（A Brief History of Time）- 史蒂芬·霍金高原上的生活（Tuesdays with Morrie）- 米奇·阿尔班大鱼（Big Fish）- 丹尼尔·沃勒斯世界尽头与冷酷仙境（Hard-Boiled Wonderland and the End of the World）- 村上春树美国众神（American Gods）- 尼尔·盖曼这些建议的书籍各具特色，应该会满足您对多种类型书籍的

### 垂直类的问题

In [29]:
import pandas as pd
benchdf_specialized = pd.read_csv("./z-bench/specialized.samples.csv")
benchdf_specialized.shape

(39, 5)

In [31]:
import random
r = random.randint(0, 39)
test_data = benchdf_specialized.iloc[r]

import random
r = random.randint(0, len(benchdf_specialized))
test_data = benchdf_emergent.iloc[r]


payload = pg.create_payload(test_data["Prompt"], model_code)
response = query_endpoint(json.dumps(payload).encode('utf-8'))

print("任务类型:", test_data["任务类型"], "参考回答:", test_data["参考回答"])
print('Prompt:', test_data["Prompt"])
print(model_code, response)
print("GPT3.5:", test_data["GPT3.5"])
print("GPT4:", test_data["GPT4"])


任务类型: `时代错误判断` 参考回答: 存在时代错误
Prompt: 请告诉我以下陈述是否包含时代错误：在盟军轰炸硫磺岛海滩期间，拉尔夫大声地对着自己的 iPhone 说话。
cpmbee [{'question': '请告诉我以下陈述是否包含时代错误：在盟军轰炸硫磺岛海滩期间，拉尔夫大声地对着自己的 iPhone 说话。', '<ans>': '是'}]
GPT3.5: 这个陈述包含时代错误，因为在二战期间还没有 iPhone 手机。
GPT4: 是的，这个陈述包含了一个时代错误。盟军轰炸硫磺岛（Iwo Jima）的事件发生在1945年的第二次世界大战期间，而iPhone是在2007年首次发布的。所以，在硫磺岛战役期间，人们不可能拥有或使用iPhone。
